In [1]:
import tabula
from pypdf import PdfReader
import re
import pandas as pd
import numpy as np

In [101]:
# 목차에서 검색해서 지금 챕터의 인덱스와 다음 챕터의 이름, 인덱스를 리턴, 못 찾았으면 -1을 리턴
def title_to_page_index(reader, chapter_title):
    length_of_table_of_contents = get_first_page_index(reader)
    for idx in range(length_of_table_of_contents):
        page = reader.pages[idx]
        table_of_contents = page.extract_text()
        start_index = table_of_contents.find(chapter_title)
        if start_index != -1:
            end_of_line = table_of_contents[start_index:].find("\n") + start_index
            page_index = int(table_of_contents[start_index:end_of_line].split(". ")[-1]) + length_of_table_of_contents - 1      # 0부터 시작하는 인덱스니까 -1을 넣어줌
            r = re.compile(r"[.][ ]?[^.]+[ ]?[.]{3}")
            match = r.search(table_of_contents[end_of_line+1:]).span()
            chapter_title_of_next_chapter = table_of_contents[match[0]+(end_of_line+1):match[1]+(end_of_line+1)].strip(" .")
            end_of_next_line = table_of_contents[end_of_line+1:].find("\n") + (end_of_line+1)
            page_index_of_next_chapter = int(table_of_contents[end_of_line+1:end_of_next_line].split(". ")[-1]) + length_of_table_of_contents - 1
            result = {
                "chapter_title" : chapter_title,
                "page_index": page_index,
                "chapter_title_of_next_chapter" : chapter_title_of_next_chapter,
                "page_index_of_next_chapter" : page_index_of_next_chapter
            }
            #print(result)
            return result
    return -1

# 첫 페이지의 페이지 인덱스를 리턴
def get_first_page_index(reader):
    for idx, page in enumerate(reader.pages):
        text = page.extract_text()
        if text.rfind("Page 1") != -1:
            return idx

# 챕터의 텍스트를 리턴
def get_text_of_chapter(reader, chapter_title):
    number_of_pages = len(reader.pages)
    page_info = title_to_page_index(reader, chapter_title)
    start_page = page_info["page_index"] - 1 if page_info["page_index"] > 0 else page_info["page_index"]
    end_page = page_info["page_index_of_next_chapter"] + 1 if page_info["page_index_of_next_chapter"] < (number_of_pages - 1) else page_info["page_index_of_next_chapter"]  # 마지막 페이지를 넘어가지 않도록 조건 지정
    text = ""
    for page_number in range(start_page, end_page + 1):
        page = reader.pages[page_number]
        text = "".join([text, page.extract_text()])
    r = re.compile(f"(?<!\. ){page_info['chapter_title']}")
    start = r.search(text).start()
    r = re.compile(f"(?<!\. ){page_info['chapter_title_of_next_chapter']}")
    end = r.search(text).start()
    return text[start:end]

In [102]:
reader = PdfReader("data/hyundai.pdf")
get_text_of_chapter(reader, "연결 재무상태표")

'연결 재무상태표\n제 56 기          2023.12.31 현재\n제 55 기          2022.12.31 현재\n제 54 기          2021.12.31 현재\n(단위 : 백만원)\n\u3000 제 56 기 제 55 기 제 54 기\n자산 \u3000 \u3000 \u3000\n\u3000유동자산 101,724,717 96,389,273 88,565,366\n\u3000\u3000현금및현금성자산 19,166,619 20,864,879 12,795,554\n\u3000\u3000단기금융상품 7,339,968 5,774,597 6,949,333\n\u3000\u3000기타금융자산 2,802,611 5,934,745 12,396,646\n\u3000\u3000매출채권 4,682,182 4,279,057 3,147,296\n\u3000\u3000기타채권 3,431,169 4,458,689 4,220,970\n\u3000\u3000재고자산 17,400,346 14,291,216 11,645,641\n\u3000\u3000기타자산 3,150,939 2,640,553 2,081,853\n\u3000\u3000당기법인세자산 195,696 85,867 47,346\n\u3000\u3000금융업채권 43,120,684 38,037,368 35,252,606\n\u3000\u3000매각예정비유동자\n산434,503 22,302 28,121\n\u3000비유동자산 180,738,638 159,353,189 145,381,049\n\u3000\u3000장기금융상품 154,966 112,557 306,410\n\u3000\u3000기타금융자산 4,423,388 3,889,776 3,539,286\n\u3000\u3000장기성매출채권 210,979 179,781 137,157\n\u3000\u3000기타채권 855,015 821,050 741,168\n\u3000\u3000기타자산 1,970,119 1,550,455 1,191,284\n\u3000\u3000유형

In [117]:
# 삼성 : 80~83
# 현대 : 69~73
# 한전 : 165~167

tabula_result_dfs = tabula.read_pdf(
    "data/hyundai.pdf",
	pages="69-73",
	stream=True,
    lattice=True
)
print(f"Data Type :{type(tabula_result_dfs)}")
print(f"Data Length: {len(tabula_result_dfs)}")
for index, table in enumerate(tabula_result_dfs):
    print(f"\nData Index: {index}")
    print(type(table))
    print(table.head())

Data Type :<class 'list'>
Data Length: 7

Data Index: 0
<class 'pandas.core.frame.DataFrame'>
  Unnamed: 0       제 56 기      제 55 기      제 54 기
0         자산          NaN         NaN         NaN
1       유동자산  101,724,717  96,389,273  88,565,366
2   현금및현금성자산   19,166,619  20,864,879  12,795,554
3     단기금융상품    7,339,968   5,774,597   6,949,333
4     기타금융자산    2,802,611   5,934,745  12,396,646

Data Index: 1
<class 'pandas.core.frame.DataFrame'>
        부채  Unnamed: 0  Unnamed: 1  Unnamed: 2
0     유동부채  73,362,103  74,236,472  64,236,787
1     매입채무  10,952,046  10,797,065   9,155,255
2     미지급금   8,642,808   8,277,891   6,335,645
3    단기차입금   9,035,548  11,366,480  13,087,836
4  유동성장기부채  25,109,158  25,574,131  20,578,902

Data Index: 2
<class 'pandas.core.frame.DataFrame'>
  Unnamed: 0       제 56 기       제 55 기       제 54 기
0        매출액  162,663,579  142,151,469  116,448,159
1       매출원가  129,179,183  113,879,569   94,721,313
2      매출총이익   33,484,396   28,271,900   21,726,846
3    판매비와관

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [118]:
year_regular_expr = re.compile("제 [0-9]+ 기")
columns = []
dfs = []
for tabula_result_df in tabula_result_dfs[0:6]:
    tabula_result_df_columns = [item for item in tabula_result_df.columns]
    if year_regular_expr.match(tabula_result_df_columns[-1]):
        columns = tabula_result_df_columns
        dfs.append(tabula_result_df)
    else:
        temp_df = pd.DataFrame(tabula_result_df_columns, index = columns).T
        dfs[-1] = pd.concat([dfs[-1], temp_df], ignore_index=True)
        tabula_result_df.columns = columns
        dfs[-1] = pd.concat([dfs[-1], tabula_result_df], ignore_index=True)
for df in dfs:
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x : x.replace("\r", "") if x is not np.nan else x)

In [119]:
dfs[0]

,Unnamed: 0,제 56 기,제 55 기,제 54 기
0,자산,NaN,NaN,NaN
1,유동자산,"101,724,717","96,389,273","88,565,366"
2,현금및현금성자산,"19,166,619","20,864,879","12,795,554"
3,단기금융상품,"7,339,968","5,774,597","6,949,333"
4,기타금융자산,"2,802,611","5,934,745","12,396,646"
...,...,...,...,...
56,기타포괄손익누계액,"(838,892)","(1,620,682)","(1,772,601)"
57,이익잉여금,"88,665,805","79,953,601","73,167,855"
58,비지배지분,"9,312,129","8,547,258","7,629,667"
59,자본총계,"101,809,440","90,896,545","82,615,789"


In [114]:
len(dfs)

2

### 우선 데이터프레임의 제목이 뭔지 알아야 함.
### 그 다음 \u3000 갯수를 보고 부모 계층을 음... _ (언더바) 로 구분해서 앞에넣기

In [117]:
""" ㅋㅋㅋ lattice = True 주니까 이거 할 필요 없어졌음
title_string = ""
edge_thickness = 0
edge_ascending = True
new_df = []
new_row = []
for idx, row in dfs[1].iterrows():
    print(idx)
    row_values = dfs[1].iloc[idx, 1:]
    row_title = dfs[1].iat[idx, 0]
    if row_values.isnull().sum() == len(row_values):
        if edge_ascending == True:
            edge_thickness += 1
            title_string = "".join([title_string, row_title])
            print(f"edge_ascending : {edge_ascending}, edge_thickness : {edge_thickness}, title_string : {title_string}")
        else:
            edge_thickness -= 1
            title_string = "".join([title_string, row_title])
            print(f"edge_ascending : {edge_ascending}, edge_thickness : {edge_thickness}, title_string : {title_string}")
        #print(dfs[1].iat[idx,0], "All Null")
    else:                                   # Null이 아닌 값이 채워져 있으면
        if edge_thickness > 0:
            edge_ascending = False
        title_string = "".join([title_string, row_title])
        contents = row_values
    if edge_thickness == 0:
        new_row.append(title_string)
        new_row.extend(contents.values)
        new_row = pd.Series(new_row, index = row.index)
        new_df.append(new_row)
        new_row = []
        edge_ascending = True
        title_string = ""
        #print(dfs[1].iat[idx,0], "values")
"""

0
1
2
edge_ascending : True, edge_thickness : 1, title_string : 후속적으로 당기손익
3
4
edge_ascending : False, edge_thickness : 0, title_string : 후속적으로 당기손익으로 재분류되지 않는항목
5
edge_ascending : True, edge_thickness : 1, title_string : 기타포괄손익-공정
6
7
edge_ascending : False, edge_thickness : 0, title_string : 기타포괄손익-공정가치측정금융자산관련손익
8
edge_ascending : True, edge_thickness : 1, title_string : 확정급여제도의 재
9


TypeError: sequence item 1: expected str instance, float found

#### trim돼서 계층을 알 수 없으니, pypdf로 나온 값을 참고해서, 맨 왼쪽꺼는 \n 다음에 \u3000이 몇번 나왔는지로 계층을 판단하자.
#### 그리고 현대꺼 줄바꿈 주의..